## Library Import

In [1]:
import pandas as pd
import plotly.express as px

## Data import

In [7]:
df = pd.read_csv("data/ulti-dataDict.csv")

df["exp-id"] = df["exp-id"].astype(str)

unique_studies = df.drop_duplicates(subset=["exp-id"])

print(df.columns)

df.head(5)

Index(['exp-id', 'paper', 'authors', 'institutes', 'city', 'country',
       'climate-class', 'pub-year', 'pub-month', 'data-avail', 'exp-months',
       'exp-type', 'function', 'timing-type', 'sample-size-calc',
       'sample-size-calc-type', 'part-payment', 'part-no-tot',
       'fem-total-ratio', 'age-group', 'bmi-range', 'normalisation-length',
       'session-length', 'tested-t-min', 'tested-t-max', 'physio-parameter',
       'physio-sensor-type', 'physio-body-site', 'physio-body-site-2',
       'skin-body-site-code', 'physio-sensor-brand', 'physio-sensor-model',
       'physio-mst-calc', 'environment-parameter',
       'environment-sensor-vert-loc', 'feedback-quest-type', 'feedback-scales',
       'cognitive-test-type', 'pub-type', 'pub-name', 'publisher', 'doi',
       'protocol-fixed-clo', 'protocol-observed-clo',
       'protocol-avoid-stimulant', 'protocol-avoid-activity',
       'protocol-avoid-heavy-food', 'protocol-rest-sleep',
       'protocol-circadian', 'protocol-mens-

,exp-id,paper,authors,institutes,city,country,climate-class,pub-year,pub-month,data-avail,...,part-meta-personality,part-meta-psych-eval,part-meta-smoking,part-meta-activity-level,part-meta-health-level,part-meta-morningness,part-meta-bmr,part-meta-alcohol-use,part-meta-mens-timing,part-meta-contraceptive-type
0,8,Association between thermal response and endog...,"Mengyuan He, Songtao Hu, Mingli Lu, Rujin Liu,...","Qingdao University of Technology, Qingdao Inte...",Qingdao,China,NaN,2023,February,Data not available for sharing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,Association between thermal response and endog...,"Mengyuan He, Songtao Hu, Mingli Lu, Rujin Liu,...","Qingdao University of Technology, Qingdao Inte...",Qingdao,China,NaN,2023,February,Data not available for sharing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,Association between thermal response and endog...,"Mengyuan He, Songtao Hu, Mingli Lu, Rujin Liu,...","Qingdao University of Technology, Qingdao Inte...",Qingdao,China,NaN,2023,February,Data not available for sharing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Association between thermal response and endog...,"Mengyuan He, Songtao Hu, Mingli Lu, Rujin Liu,...","Qingdao University of Technology, Qingdao Inte...",Qingdao,China,NaN,2023,February,Data not available for sharing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Association between thermal response and endog...,"Mengyuan He, Songtao Hu, Mingli Lu, Rujin Liu,...","Qingdao University of Technology, Qingdao Inte...",Qingdao,China,NaN,2023,February,Data not available for sharing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Contextual data

#### Studies by country

In [3]:
country_counts = unique_studies["country"].value_counts().reset_index()
country_counts.columns = ["country", "count"]

fig = px.choropleth(
    country_counts,
    locations="country",
    locationmode="country names",
    color="count",
    width=800,
    height=600,
    hover_name="country",
)

fig.show()

#### Studies per year

In [4]:
year_counts = unique_studies["pub-year"].value_counts().sort_index().reset_index()
year_counts.columns = ["pub-year", "count"]

fig = px.line(
    year_counts,
    x="pub-year",
    y="count",
    title="Number of Studies Published Each Year",
    width=800,
    height=400,
    template="plotly_white",
)

fig.show()

In [16]:
fig = px.bar(
    year_counts, x="pub-year", y="count", width=800, height=400, template="plotly_white"
)

fig.show()

#### Studies by building type

In [5]:
function_counts = df["function"].value_counts().reset_index()
function_counts.columns = ["function", "count"]

fig = px.pie(
    function_counts,
    names="function",
    values="count",
    width=600,
    height=400,
    template="plotly_white",
)

fig.show()

## Physiological Parameters

#### Recorded parameters

The most **recorded parameter** is skin temperature, followed by heart rate. Bloop pressure, sweat, blood oxygen saturation and EEG are not recorded as often.

In [9]:
##
parameter_counts = df.groupby("physio-parameter")["exp-id"].nunique()
total_experiments = df["exp-id"].nunique()

parameter_counts = parameter_counts.reset_index(name="exp-id count")
parameter_counts["exp-id percentage"] = (
    parameter_counts["exp-id count"] / total_experiments
) * 100
parameter_counts_sorted = parameter_counts.sort_values(
    "exp-id percentage", ascending=False
)

fig = px.bar(
    parameter_counts_sorted,
    x="physio-parameter",
    y="exp-id percentage",
    labels={
        "physio": "Physiological Parameter [-]",
        "exp-id percentage": "Percentage of Studies",
    },
)

fig.update_layout(
    xaxis_title="Physiological Parameter [-]",
    yaxis_title="Percentage of Studies [%]",
    template="plotly_white",
)

fig.show()

#### Skin temperature: Locations and calculation

**Skin temperature locations** and **calculations**

In [13]:
skin_temperature = df[df["physio-parameter"] == "Skin Temperature"]
location_counts = skin_temperature["physio-body-site"].value_counts()

fig = px.bar(
    location_counts,
)

fig.update_layout(
    xaxis_title="Measurement location [-]",
    yaxis_title="Percentage of Studies [%]",
    template="plotly_white",
)

fig.show()

In [20]:
print(location_counts)

physio-body-site
Forehead     3
Upper arm    3
Chest        3
Thigh        3
Hand         2
Calf         2
Back         1
Forearm      1
Scapula      1
Wrist        1
Name: count, dtype: int64


In [140]:
import plotly.graph_objects as go

testing_location_counts = pd.DataFrame(
    {
        "physio-body-site": [
            "Forehead",
            "Nose",
            "Cheek",
            "Neck",
            "Chest",
            "Back",
            "Upper arm",
            "Abdomen",
            "Lumbar",
            "Forearm",
            "Buttock",
            "Wrist",
            "Finger",
            "Thigh",
            "Shin",
            "Calf",
            "Ankle",
            "Foot",
            "Sole",
        ],
        "count": [10, 15, 7, 20, 5, 8, 12, 14, 9, 11, 6, 13, 4, 16, 3, 18, 2, 17, 1],
    }
)

location_coordinates = pd.DataFrame(
    {
        # Format: 'physio-body-site':     [x, y]
        "Forehead": [-147.5, 262.5],
        "Nose": [-147.5, 240],
        "Cheek": [-160, 225],
        "Neck": [28, 203],
        "Chest": [-147.5, 157.5],
        "Back": [28, 152.5],
        "Upper arm": [-195, 140],
        "Abdomen": [-147.5, 75],
        "Lumbar": [28, 85],
        "Forearm": [-195, 75],
        "Buttock": [45, 55],
        "Wrist": [-205, 45],
        "Finger": [-215, -5],
        "Thigh": [28, -22.5],
        "Shin": [-170, -110],
        "Calf": [12.5, -110],
        "Ankle": [17.5, -165],
        "Foot": [-150, -190],
        "Sole": [17.5, -190],
    }
)

location_coordinates = location_coordinates.transpose().rename(
    columns={0: "x-coordinate", 1: "y-coordinate"}
)
location_coordinates.reset_index(inplace=True)
location_coordinates.rename(columns={"index": "physio-body-site"}, inplace=True)

merged_df = pd.merge(
    location_coordinates, testing_location_counts, on="physio-body-site"
)


fig = go.Figure()

fig.add_layout_image(
    dict(
        source="assets/img/body-site-chart-background.png",
        xref="x",
        yref="y",
        x=10,
        y=10,
        sizex=800,
        sizey=640,
        xanchor="center",
        yanchor="middle",
        # sizing="stretch",
        opacity=1.0,
        layer="below",
    )
)

fig.add_trace(
    go.Scatter(
        x=merged_df["x-coordinate"],
        y=merged_df["y-coordinate"],
        mode="markers",
        marker=dict(size=merged_df["count"] * 2.5, color="#db1492", opacity=0.8),
        text=merged_df["count"],  # Add the counts to the hover text
        hoverinfo="text",
    )
)

fig.update_xaxes(showgrid=False, showticklabels=False, range=[-350, 350])

fig.update_yaxes(showgrid=False, showticklabels=False, range=[-320, 320])

fig.update_layout(
    plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    width=800,
    height=640,
    margin=dict(l=10, r=10, t=10, b=10),
)


fig.show()

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('int64')) -> None

#### Skin temperature: Reasoning

**Mentioning of reasoning** for selected locations and equations (Y/N)

#### Heart Rate: Sensors and calculation

**Heart rate sensors** and **locations**

In [10]:
heart_rate = df[df["physio-parameter"] == "Heart Rate"]
heart_rate_location_counts = heart_rate["how"].value_counts().reset_index()
heart_rate_location_counts.columns = ["Location", "Count"]

fig = px.pie(
    heart_rate_location_counts,
    names="Location",
    values="Count",
    width=600,
    height=400,
    template="plotly_white",
)

fig.show()

#### Additional

In [11]:
fig = px.sunburst(
    df, path=["physio", "how", "where"], width=800, height=600, template="plotly_white"
)

fig.show()

---

## Protocols

#### Session and acclimatisation length
NOTE: Non-existing normalisation should be counted as zero

In [29]:
fig = px.box(
    unique_studies,
    y=["session-length", "normalisation-length"],
    width=800,
    height=400,
    template="plotly_white",
)

fig.update_layout(
    xaxis_title=None,
    yaxis_title="Time [min]",
)

fig.show()

ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [13]:
fig = px.scatter(
    df,
    x="tested-t-min",
    y="tested-t-max",
    color="physio",
    size="part-no-tot",  # replace with something else
    width=800,
    height=400,
    template="plotly_white",
)

fig.update_yaxes(
    range=[0, 100],
)

fig.show()

#### Time of the day

**Time of the day**: Start/end times & circadian rhythm

In [14]:
# maybe something with a dial/clock plot for time of day

df.columns

Index(['exp-id', 'paper', 'authors', 'institute', 'city', 'country',
       'pub-year', 'exp-type', 'function', 'timing-type', 'sample-size-calc',
       'sample-size-calc-type', 'part-no-tot', 'fem-total-ratio', 'age-group',
       'bmi-range', 'selection-inclusion', 'normalisation-length',
       'session-length', 'data-avail', 'tested-t-min', 'tested-t-max',
       'physio', 'how', 'where', 'environment', 'feedback', 'cognitive',
       'pub-type', 'pub-name', 'doi'],
      dtype='object')

#### Control Contextual Parameters

**Control of contextual parameters**: caffeine, alcohol, smoking, medication, exercise

In [ ]:
# heatmap or stacked bar chart

#### Comfort questionnaires

**Thermal comfort questionnaires**

## Background Thermal Environment

#### Measure parameters

Measured parameters, e.g. air temperature, relative humidity, etc.

In [15]:
fig = px.parallel_categories(
    df,
    dimensions=["exp-id", "function", "environment"],
    width=800,
    height=400,
    template="plotly_white",
)

fig.show()

#### Positioning

Information on positioning

---

## Participants

#### Age Group

In [16]:
fig = px.pie(
    unique_studies,
    names="age-group",
    title="Age Group Distribution",
    width=800,
    height=400,
    template="plotly_white",
)

fig.show()

#### Female-Male Ratio

In [30]:
fig = px.violin(
    unique_studies,
    y="fem-total-ratio",
    # box=True,
    # points="all",
    title="Gender Ratio in Studies",
    width=800,
    height=400,
    template="plotly_white",
)

fig.show()

#### No. Participants

In [31]:
fig = px.box(
    unique_studies,
    y="part-no-tot",
    title="Total Number of Participants in Studies",
    width=800,
    height=400,
    template="plotly_white",
)

fig.show()

#### Sample Size

---

## Data Processing and Availability

#### Data availability

In [32]:
fig = px.bar(
    unique_studies,
    x="data-avail",
    width=800,
    height=400,
    title="Data Availability in Studies",
    template="plotly_white",
)

fig.show()

#### Data processing

## Misc

In [33]:
fig = px.histogram(
    unique_studies,
    x="exp-type",
    facet_col="country",
    width=800,
    height=400,
    title="Experiment Types by Country",
    template="plotly_white",
)

fig.show()